# Wess Zumino 

## N=1 in 1+1

The Hamiltonian with a central difference for the derivative is given by 

\begin{align}
    H=\sum_n&\left[ \frac{p_n^2}{2a} + \frac{a}{2}\left(\frac{\phi_{n+1}-\phi_{n-1}}{2a}\right)^2+\frac{a}{2}V(\phi_n)^2
            +aV(\phi_n)\frac{\phi_{n+1}-\phi_{n-1}}{2a} \right. \nonumber \\
            &\left.+(-1)^nV'(\phi_n)\left(\chi_n^{\dagger}\chi_n-\frac{1}{2}\right)
            +\frac{1}{2a}\left(\chi_n^{\dagger}\chi_{n+1}+\chi_{n+1}^{\dagger}\chi_n\right) \right],
\end{align}

In [1]:
import sympy as sp
from ham_to_matrix import *
from constants import *


In [2]:
bosonNI=sp.expand( pn**2/(2*aLat) + (aLat/2)*((qnP1-qnM1)/(2*aLat))**2 )
bosonI=sp.expand( (aLat/2)*V(qn)**2 + aLat*V(qn)*(qnP1-qnM1)/(2*aLat) )
fermionNI=sp.expand( (xdnP1*xn+xdn*xnP1)/(2*aLat) )
fermionI=sp.expand( (-1)**n*sp.diff(V(qn),qn)*(xdn*xn-(1/2)) )

In [3]:
bosonNI+bosonI

a*V(q_{n})**2/2 + V(q_{n})*q_{n+1}/2 - V(q_{n})*q_{n-1}/2 + p_{n}**2/(2*a) - q_{n+1}*q_{n-1}/(8*a) + q_{n+1}**2/(8*a) - q_{n-1}*q_{n+1}/(8*a) + q_{n-1}**2/(8*a)

In [4]:
fermionNI + fermionI

-0.5*(-1)**n*Derivative(V(q_{n}), q_{n}) + (-1)**n*Derivative(V(q_{n}), q_{n})*\chi^{\dagger}_{n}*\chi_{n} + \chi^{\dagger}_{n+1}*\chi_{n}/(2*a) + \chi^{\dagger}_{n}*\chi_{n+1}/(2*a)

In [5]:
N=3
aVal=1

# depends on finite-difference method
qs=[SiteSymbol('q',str(i)) for i in range(-1,N+1,1)]
# maybe make qs a normal site list
# and make an extra boundaryQs list for q[-1], q[N]
# counting would be normal computer science way for rest of code.

ps=[SiteSymbol('p',str(i)) for i in range(-1,N+1,1)] # don't really need extras
aops=[SiteSymbol('a',str(i)) for i in range(-1,N+1,1)]
adags=[SiteSymbol('a^{\dagger}',str(i)) for i in range(-1,N+1,1)]
xs=[SiteSymbol('\chi',str(i)) for i in range(-1,N+1,1)]
xdags=[SiteSymbol('\chi^{\dagger}',str(i)) for i in range(-1,N+1,1)]

# note this is exactly hardcoded for this finite difference method.
bcType = 'periodic'
boundaryConditions = {}
if bcType == 'periodic':
    boundaryConditions = {qs[0]: qs[N], qs[N+1]: qs[1],
                          xs[0]: -xs[N], xs[N+1]: -xs[1], 
                          xdags[0]: -xdags[N],  xdags[N+1]: -xdags[1]
                         }
    
elif bcType == 'dirichlet':
    boundaryConditions = { qs[0]: 0, qs[N+1]: 0,
                          xs[0]: 0, xs[N+1]: 0,
                          xdags[0]: 0, xdags[N+1]:0
                         }

In [6]:
totHam=bosonNI+bosonI+fermionNI+fermionI
totHam

-0.5*(-1)**n*Derivative(V(q_{n}), q_{n}) + (-1)**n*Derivative(V(q_{n}), q_{n})*\chi^{\dagger}_{n}*\chi_{n} + a*V(q_{n})**2/2 + V(q_{n})*q_{n+1}/2 - V(q_{n})*q_{n-1}/2 + \chi^{\dagger}_{n+1}*\chi_{n}/(2*a) + \chi^{\dagger}_{n}*\chi_{n+1}/(2*a) + p_{n}**2/(2*a) - q_{n+1}*q_{n-1}/(8*a) + q_{n+1}**2/(8*a) - q_{n-1}*q_{n+1}/(8*a) + q_{n-1}**2/(8*a)

In [7]:
ham=0

for i in range(1,N+1):
    ham+=(bosonNI+bosonI+fermionNI).subs({
        pn: ps[i],
        qn: qs[i], qnP1: qs[i+1], qnM1: qs[i-1],
        xn: xs[i], xnP1: xs[i+1],
        xdn: xdags[i], xdnP1: xdags[i+1]
    }).subs(boundaryConditions)
    
# ham.subs(boundaryConditions).doit() # this doesn't work?


def potential(n):
    # m*q with m=1
    return qs[n]

potentialSubs={}
for n in range(1,N+1):
    potentialSubs[V(qs[n])]=potential(n)

ham=sp.simplify(ham.subs(potentialSubs).subs(aLat,aVal))


for i in range(1,N+1):
    if i%2==0:
        ham+=sp.simplify(fermionI.subs({qn: qs[i], xn: xs[i], xdn: xdags[i]}).subs(potentialSubs)).subs((-1)**n,1)
    else:
        ham-=sp.simplify(fermionI.subs({qn: qs[i], xn: xs[i], xdn: xdags[i]}).subs(potentialSubs)).subs((-1)**n,1)
ham=sp.simplify(ham)
ham

(4.0 - 8*\chi^{\dagger}_{0}*\chi_{0} + 4*\chi^{\dagger}_{0}*\chi_{1} - 4*\chi^{\dagger}_{0}*\chi_{2} + 4*\chi^{\dagger}_{1}*\chi_{0} + 8*\chi^{\dagger}_{1}*\chi_{1} + 4*\chi^{\dagger}_{1}*\chi_{2} - 4*\chi^{\dagger}_{2}*\chi_{0} + 4*\chi^{\dagger}_{2}*\chi_{1} - 8*\chi^{\dagger}_{2}*\chi_{2} + 4*p_{0}**2 + 4*p_{1}**2 + 4*p_{2}**2 + 3*q_{0}*q_{1} - 5*q_{0}*q_{2} + 6*q_{0}**2 - 5*q_{1}*q_{0} + 3*q_{1}*q_{2} + 6*q_{1}**2 + 3*q_{2}*q_{0} - 5*q_{2}*q_{1} + 6*q_{2}**2)/8

In [8]:
#ham=0
#totHam=bosonNI+bosonI+fermionNI+fermionI
#for i in range(0,N):
#    ham+=totHam.subs(n,i).subs(boundaryConditions)
# ham.subs(boundaryConditions).doit() # this doesn't work?

#def potential(n):
#    return q[n]

#for n in range(0,N):
#    ham=ham.subs(V(q[n]),potential(n))
#ham=ham.subs(aLat,aVal).simplify().expand()
#ham

In [9]:
m=1

HOdofSubs = {}
#offset because of BC
for i in range(1,N+1):
    HOdofSubs[qs[i]] = 0.5*sp.sqrt(2/m)*(aops[i] + adags[i])
    HOdofSubs[ps[i]] = complex(0,1)*sp.sqrt(2*m)*(adags[i] - aops[i])/2 

hoHam=sp.expand(ham.subs(HOdofSubs))
hoHam=sp.nsimplify(hoHam,tolerance=1e-8)
hoHam

1/2 - \chi^{\dagger}_{0}*\chi_{0} + \chi^{\dagger}_{0}*\chi_{1}/2 - \chi^{\dagger}_{0}*\chi_{2}/2 + \chi^{\dagger}_{1}*\chi_{0}/2 + \chi^{\dagger}_{1}*\chi_{1} + \chi^{\dagger}_{1}*\chi_{2}/2 - \chi^{\dagger}_{2}*\chi_{0}/2 + \chi^{\dagger}_{2}*\chi_{1}/2 - \chi^{\dagger}_{2}*\chi_{2} + 3*a^{\dagger}_{0}*a^{\dagger}_{1}/16 - 5*a^{\dagger}_{0}*a^{\dagger}_{2}/16 + 5*a^{\dagger}_{0}*a_{0}/8 + 3*a^{\dagger}_{0}*a_{1}/16 - 5*a^{\dagger}_{0}*a_{2}/16 + a^{\dagger}_{0}**2/8 - 5*a^{\dagger}_{1}*a^{\dagger}_{0}/16 + 3*a^{\dagger}_{1}*a^{\dagger}_{2}/16 - 5*a^{\dagger}_{1}*a_{0}/16 + 5*a^{\dagger}_{1}*a_{1}/8 + 3*a^{\dagger}_{1}*a_{2}/16 + a^{\dagger}_{1}**2/8 + 3*a^{\dagger}_{2}*a^{\dagger}_{0}/16 - 5*a^{\dagger}_{2}*a^{\dagger}_{1}/16 + 3*a^{\dagger}_{2}*a_{0}/16 - 5*a^{\dagger}_{2}*a_{1}/16 + 5*a^{\dagger}_{2}*a_{2}/8 + a^{\dagger}_{2}**2/8 + 5*a_{0}*a^{\dagger}_{0}/8 + 3*a_{0}*a^{\dagger}_{1}/16 - 5*a_{0}*a^{\dagger}_{2}/16 + 3*a_{0}*a_{1}/16 - 5*a_{0}*a_{2}/16 + a_{0}**2/8 - 5*a_{1}*a^{\da

In [10]:
cutoff=4
term=hoHam.args[1]
hamMat=convert_term_to_matrix(term,cutoff,N,aops,adags,xs,xdags)

setup timer took: 0.0133 seconds
a^{\dagger}_{0}**2setup timer took: 0.0006 seconds
a^{\dagger}_{0}**2full timer took: 11.1048 seconds
a^{\dagger}_{0}**2multiply timer took: 0.0074 seconds


In [11]:
t=Timer('test')
t.start()
np.kron(np.random.rand((4**4)*(2**4),(4**4)*(2**4)),np.random.rand(4,4))
t.stop()

test timer took: 3.9318 seconds


In [12]:
(4**3)*(2**3)

512

In [13]:
t=Timer('test')
t.start()
np.kron(np.random.rand((4**3)*(2**3),(4**3)*(2**3)),np.random.rand(2,2))
t.stop()

test timer took: 0.0249 seconds


In [14]:
t=Timer('test')
t.start()
np.kron(np.random.rand((5**3)*(2**3),(5**3)*(2**3)),np.random.rand(2,2))
t.stop()

test timer took: 0.0744 seconds


In [15]:
t=Timer('test')
t.start()
np.kron(np.random.rand((6**3)*(2**3),(6**3)*(2**3)),np.random.rand(2,2))
t.stop()

test timer took: 0.2160 seconds


In [16]:
t=Timer('test')
t.start()
#crashes
#np.kron(np.random.rand((6**4)*(2**4),(6**4)*(2**4)),np.random.rand(2,2))
t.stop()

test timer took: 0.0000 seconds
